In [1]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

AttributeError: module 'tensorflow' has no attribute '__version__'

In [ ]:
print(tf.__version__,tfdf.__version__)

In [2]:
dataset_df = pd.read_csv('/data/train.csv')
print(dataset_df.shape)

NameError: name 'pd' is not defined

In [ ]:
dataset_df.head(5)

In [ ]:
dataset_df.describe()

In [ ]:
dataset_df.columns

In [ ]:
plot_df = dataset_df.diagnosed_diabetes.value_counts()
plot_df.plot(kind='bar')

In [ ]:
def plot_funtion(lenth):
    fig,ax = plt.subplots(lenth,1,figsize=(10,2*lenth))
    plt.subplots_adjust(top=2)
    for i in range(lenth):
        sns.histplot(dataset_df[dataset_df.columns[i]],color='b',bins=50,ax=ax[i])


plot_funtion(len(dataset_df.columns))

In [ ]:
dataset_df.columns

In [ ]:
dataset_df = dataset_df.drop(['id'],axis=1)
dataset_df.head(5)

In [ ]:
dataset_df.isnull().sum().sort_values(ascending=False)

In [ ]:
def split_dataset(dataset, test_ratio=0.20):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(valid_ds_pd)))

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label='diagnosed_diabetes')
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label='diagnosed_diabetes')

In [ ]:
rf = tfdf.keras.RandomForestModel()
rf.compile(metrics=["accuracy"])

In [ ]:
rf.fit(x=train_ds)

In [ ]:
import matplotlib.pyplot as plt
logs = rf.make_inspector().training_logs()
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")
plt.show()

In [ ]:
inspector = rf.make_inspector()
inspector.evaluation()

In [ ]:
evaluation = rf.evaluate(x=valid_ds,return_dict=True)

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

In [ ]:
print(f"Available variable importances:")
for importance in inspector.variable_importances().keys():
  print("\t", importance)

In [ ]:
inspector.variable_importances()["NUM_AS_ROOT"]

In [ ]:
test_df = pd.read_csv('/data/test.csv')
test_df.info()

In [ ]:
test_df = pd.read_csv('/data/test.csv')
submission_id = test_df.id


# Convert pd dataframe to tf dataset
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df)

# Get the predictions for testdata
predictions = rf.predict(test_ds)
n_predictions = (predictions > 0.5).astype(bool)
output = pd.DataFrame({'id': submission_id,
                       'diagnosed_diabetes': n_predictions.squeeze()})

output.head()

In [ ]:
sample_submission_df = pd.read_csv('/data/sample_submission.csv')
sample_submission_df['diagnosed_diabetes'] = n_predictions
sample_submission_df.to_csv('/data/submission.csv', index=False)
sample_submission_df.head()